In [1]:
# %pip install aiohttp
# %pip install mysql-connector-python
# %pip install nest_asyncio
# %pip install lenskit --upgrade
# %pip install psycopg2

In [71]:
import asyncio
import aiohttp
import json
import sqlite3
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
import urllib
from pandas.io import sql
from datetime import datetime
import numpy as np
import requests
from time import perf_counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nest_asyncio
nest_asyncio.apply()

# Performance Testing

In [72]:
class ConfigReader:
    def get_value(self, key):
        with open('config.json') as json_data_file:
            data = json.load(json_data_file)
        return data[key]

In [73]:
class DbManager:
    def __init__(self):
        reader = ConfigReader()
        db_connection = reader.get_value("db_connection")        
        self.conn_string = '{db_engine}{connector}://{user}:{password}@{server}/{database}'.format(
            db_engine=db_connection['db_engine'],
            connector=db_connection['connector'],
            user=db_connection['user'],
            password=db_connection['password'],
            server=db_connection['server'],
            database=db_connection['database'])

    def get_users(self):
        return sql.read_sql("SELECT distinct user FROM rating;", create_engine(self.conn_string))

## Get random users

In [74]:
n_rand_users = num_requests = 10
dbManager = DbManager()
db_users = dbManager.get_users()
n_rand_users = db_users.sample(n=n_rand_users)

## Test recommendation endpoint

In [75]:
base_url = 'http://127.0.0.1:8000'
n_recs = 5
items = "10,20,30,40,50"
pred_algos = ['bias','itemitem','useruser','biasedmf','implicitmf','funksvd']
rec_algos = ['popular']

### Semaphore performance

In [76]:
import os
throughputs = []

def print_stats(times, time_taken_all, num_requests):
    print(f'Total response time: {time_taken_all}')
    print(f'Throughput (requests per second): {num_requests / time_taken_all}')
    print(f'Peak response time: {max(times)}')
    print(f'Mean response time: {np.mean(times)}')
    print(f'99 percentile: {np.quantile(times, 0.99)}')

def plot_numbers(file_name):
    resp_time_per_request = np.genfromtxt(file_name, delimiter=',')
    plt.plot(resp_time_per_request)
    plt.show()
    
def hist_numbers(file_name):
    resp_time_per_request = np.genfromtxt(file_name, delimiter=',')
    plt.hist(resp_time_per_request, bins='auto')
    plt.show()

# Predictions    
async def get_preds_sem(num_sem, algo_pred, file_name=None, add_throughput=False):
    times = []
    sem = asyncio.Semaphore(num_sem)
    tasks = []    
    num_requests = len(n_rand_users)
    print(f'Number of requests: {num_requests}')
    start_preds = perf_counter()

    async with aiohttp.ClientSession() as session:
        for idx, row in n_rand_users.iterrows():
            task = asyncio.ensure_future(get_user_preds_with_sem(row['user'], algo_pred, items, session, sem, times))
            tasks.append(task)         

        responses = await asyncio.gather(*tasks)
        time_taken_all = perf_counter() - start_preds
        print_stats(times, time_taken_all, num_requests)
        
        if file_name != None and file_name != '':
            if os.path.exists(file_name):
                os.remove(file_name)
            np.savetxt(file_name, times, delimiter=',')
        
        if add_throughput:
            throughputs.append(num_requests / time_taken_all)

async def get_user_preds_with_sem(user, algo, items, session, sem, times):
    async with sem:  # semaphore limits num of simultaneous downloads
        return await get_user_preds_sem(user, algo, items, session, times)        
        
async def get_user_preds_sem(user, algo, items, session, times):
    url = f'{base_url}/algorithms/{algo}/predictions?user_id={user}&items={items}'
    start = perf_counter()
    async with session.get(url) as resp:
        data = await resp.json()    
        time_taken = perf_counter() - start
        times.append(time_taken)
        
# Recommendations
async def get_recs_sem(num_sem, algo_rec, file_name=None, add_throughput=False):
    times = []
    sem = asyncio.Semaphore(num_sem)
    tasks = []
    num_requests = len(n_rand_users)
    print(f'Number of requests: {num_requests}')
    start_preds = perf_counter()

    async with aiohttp.ClientSession() as session:
        for idx, row in n_rand_users.iterrows():
            task = asyncio.ensure_future(get_user_recs_with_sem(row['user'], algo_rec, n_recs, session, sem, times))
            tasks.append(task)         

        responses = await asyncio.gather(*tasks)
        time_taken_all = perf_counter() - start_preds
        print_stats(times, time_taken_all, num_requests)
        
        if file_name != None and file_name != '':
            if os.path.exists(file_name):
                os.remove(file_name)
            np.savetxt(file_name, times, delimiter=',')
        
        if add_throughput:
            throughputs.append(num_requests / time_taken_all)

async def get_user_recs_with_sem(user, algo, n_recs, session, sem, times):
    async with sem:  # semaphore limits num of simultaneous downloads
        return await get_user_preds_sem(user, algo, n_recs, session, times)        
        
async def get_user_recs_sem(user, algo, n_recs, session, times):
    url = f'{base_url}/algorithms/{algo}/recommendations?user_id={user}&num_recs={n_recs}'
    start = perf_counter()
    async with session.get(url) as resp:
        data = await resp.json()    
        time_taken = perf_counter() - start
        times.append(time_taken)   

### Warm up phase

In [85]:
async def warm_up_async(current_algo=None, num_workers=24, display_logs=True):
    warm_up_user = 1
    times = []
    tasks = []
    async with aiohttp.ClientSession() as session:
        for algo in pred_algos:
            if current_algo is None or algo == current_algo:
                for w in range(num_workers * 2):
                    if display_logs:
                        print(f'Calling {algo}. Worker number: {w + 1}')
                    task = asyncio.ensure_future(get_user_preds_sem(warm_up_user, algo, items, session, times))
                    tasks.append(task)
        responses = await asyncio.gather(*tasks)

In [82]:
def warm_up(current_algo=None, num_workers=24, display_logs=True):
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(warm_up_async(current_algo, num_workers, display_logs))
    loop.run_until_complete(future)

In [84]:
warm_up(None, 4)

None
Calling bias. Worker number: 1
Calling bias. Worker number: 2
Calling bias. Worker number: 3
Calling bias. Worker number: 4
Calling bias. Worker number: 5
Calling bias. Worker number: 6
Calling bias. Worker number: 7
Calling bias. Worker number: 8
None
Calling itemitem. Worker number: 1
Calling itemitem. Worker number: 2
Calling itemitem. Worker number: 3
Calling itemitem. Worker number: 4
Calling itemitem. Worker number: 5
Calling itemitem. Worker number: 6
Calling itemitem. Worker number: 7
Calling itemitem. Worker number: 8
None
Calling useruser. Worker number: 1
Calling useruser. Worker number: 2
Calling useruser. Worker number: 3
Calling useruser. Worker number: 4
Calling useruser. Worker number: 5
Calling useruser. Worker number: 6
Calling useruser. Worker number: 7
Calling useruser. Worker number: 8
None
Calling biasedmf. Worker number: 1
Calling biasedmf. Worker number: 2
Calling biasedmf. Worker number: 3
Calling biasedmf. Worker number: 4
Calling biasedmf. Worker number:

### Gunicorn methods

In [95]:
import subprocess
import os

def get_gunicorn_master_pid():
    proc1 = subprocess.Popen(['ps', 'ax'], stdout=subprocess.PIPE)
    proc2 = subprocess.Popen(['grep', 'gunicorn'], stdin=proc1.stdout,
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    proc1.stdout.close() # Allow proc1 to receive a SIGPIPE if proc2 exits.
    out, err = proc2.communicate()
    master_id = out[:6].decode('utf-8').replace(' ', '')
    return master_id

def add_workers(n):
    master_id = get_gunicorn_master_pid()
    for i in range(n):
        os.system(f"kill -s TTIN {master_id}")
        
def remove_workers(n):
    master_id = get_gunicorn_master_pid()
    for i in range(n):
        os.system(f"kill -s TTOU {master_id}")

### Call predict and recommend from server for canonical config

#### Predictions for different algorithms

In [87]:
for algo in pred_algos:
    file_name = f'preds_{algo}_parallel_threads_8_workers_4_num_req_{num_requests}.csv'
    loop = asyncio.get_event_loop()
    print(f'Algorithm: {algo}')
    future = asyncio.ensure_future(get_preds_sem(8, algo, file_name, True))
    loop.run_until_complete(future)
#    plot_numbers(file_name)
#    hist_numbers(file_name)
    print('---------------------')
    print('')

Algorithm: bias
Number of requests: 10
Total response time: 0.45714673700058484
Throughput (requests per second): 21.87481434432115
Peak response time: 0.31431903399970906
Mean response time: 0.2425812477999898
99 percentile: 0.31350228039975264
---------------------

Algorithm: itemitem
Number of requests: 10
Total response time: 5.706992383000397
Throughput (requests per second): 1.7522364371446024
Peak response time: 3.9596992719998525
Mean response time: 3.080569376800031
99 percentile: 3.9589826475398606
---------------------

Algorithm: useruser
Number of requests: 10
Total response time: 0.5534656159998121
Throughput (requests per second): 18.067969736359185
Peak response time: 0.39621907499986264
Mean response time: 0.2914110841001275
99 percentile: 0.39601378967980966
---------------------

Algorithm: biasedmf
Number of requests: 10
Total response time: 0.6948380330004511
Throughput (requests per second): 14.39184317072855
Peak response time: 0.49662871600048675
Mean response 

#### Recommendations

In [88]:
algo_rec = 'popular'
file_name = f'recs_{algo_rec}_parallel_threads_8_workers_4_num_req_{num_requests}.csv'
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(get_recs_sem(8, algo_rec, file_name))
loop.run_until_complete(future)
#plot_numbers(file_name)
#hist_numbers(file_name)

Number of requests: 10
Total response time: 0.06893939900055557
Throughput (requests per second): 145.0549344057875
Peak response time: 0.052158723000502505
Mean response time: 0.03669173280022733
99 percentile: 0.05120062710047023


### Speedup Tests

In [96]:
throughputs = []
linear_speedup_algos = ['biasedmf', 'itemitem']

In [97]:
def call_server(file_name):
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(get_preds_sem(8, current_algo, file_name, True))
    loop.run_until_complete(future)
#    plot_numbers(file_name)
#    hist_numbers(file_name)

In [98]:
workers_config = [1, 2, 4] #, 8, 12, 16, 24]
inc_config = [1, 2, 4] #, 4, 4, 8]

In [103]:
for current_algo in linear_speedup_algos:
    i = 0
    throughputs = []
    remove_workers(3) # reduce from 4 workers to 1
    for num_workers in workers_config:
        print(f'Algo: {current_algo}, Workers: {num_workers}')
        warm_up(current_algo, num_workers, display_logs=False)
        file_name = f'preds_{current_algo}_parallel_threads_8_workers_{num_workers}_num_req_{num_requests}.csv'
        call_server(file_name)
        if (num_workers != workers_config[-1]):
            print(f'add {inc_config[i]} workers')
            add_workers(inc_config[i])
        i += 1
        print('------------------')
    throughput_file_name_workers = f'throughput_single_multiple_workers_algo_{current_algo}.csv'
    np.savetxt(throughput_file_name_workers, throughputs , delimiter=',')
    remove_workers(workers_config[-1] - 4) # remove workers to get only 4 (default config)
    print('*******************************************************')
    

Algo: biasedmf, Workers: 1
Number of requests: 10
Total response time: 1.750941180999689
Throughput (requests per second): 5.711214122161752
Peak response time: 1.4139431200001127
Mean response time: 0.9230218484998659
99 percentile: 1.4135731476400724
add 1 workers
------------------
Algo: biasedmf, Workers: 2
Number of requests: 10
Total response time: 1.6936998350001886
Throughput (requests per second): 5.904233910490336
Peak response time: 1.3419794239998737
Mean response time: 0.8791842122998788
99 percentile: 1.341489655969817
add 2 workers
------------------
Algo: biasedmf, Workers: 4
Number of requests: 10
Total response time: 0.8475143490004484
Throughput (requests per second): 11.799210257376668
Peak response time: 0.6402733830000216
Mean response time: 0.4171475012998599
99 percentile: 0.6389676171899555
------------------
*************************************************
Algo: itemitem, Workers: 1
Number of requests: 10
Total response time: 15.830987913999707
Throughput (re

#### Throughput by number of workers

In [109]:
# throughput_file_name_workers = 'throughput_single_multiple_workers.csv'
# np.savetxt(throughput_file_name_workers, throughputs , delimiter=',')

In [107]:
# throughputs_workers_from_file = np.genfromtxt(throughput_file_name_workers, delimiter=',')
# workers = [1, 2, 4, 8, 16]
# y_pos = np.arange(len(throughputs_workers_from_file))

# plt.bar(y_pos, throughputs_workers_from_file, align='center', alpha=0.5)
# plt.xticks(y_pos, workers)
# plt.ylabel('Throughput')
# plt.title('Throughput by workers')

# plt.show()

### Lenskit

In [107]:
import sys
import math
from binpickle import BinPickleFile
from pathlib import Path

directory_path = 'models'
algo_pred_lkpy = 'bias.bpk'

def load_for_shared_mem(file_name):
    full_file_name = Path(directory_path) / file_name

    binpickle_file = BinPickleFile(full_file_name, direct=True)
    model = binpickle_file.load()
    return model

def get_predictions_from_model(model, user, items):
    try:
        results = []
        df_preds = model.predict_for_user(user, items)
        for index, value in df_preds.iteritems():
            if not math.isnan(value):
                results.append({'item': index, 'score': value})
        return results
    except:
        print(f"Unexpected preds error for user: {user}, with items: {items}. Error: {sys.exc_info()[0]}")
        raise
        

# Predictions    
async def get_preds_threads_lkpy(num_sem, model, file_name=None, add_throughput=False):
    times = []
    sem = asyncio.Semaphore(num_sem)
    tasks = []    
    num_requests = len(n_rand_users)
    print(f'Number of requests: {num_requests}')
    start_preds = perf_counter()

    async with aiohttp.ClientSession() as session:
        for idx, row in n_rand_users.iterrows():
            task = asyncio.ensure_future(get_user_preds_with_threads_lkpy(row['user'], algo_pred_lkpy, items, session, sem, times, model))
            tasks.append(task)         

        responses = await asyncio.gather(*tasks)
        time_taken_all = perf_counter() - start_preds
        print_stats(times, time_taken_all, num_requests)
        
        if file_name != None and file_name != '':
            if os.path.exists(file_name):
                os.remove(file_name)
            np.asarray(times)
            np.savetxt(file_name, times, delimiter=',')
        
        if add_throughput:
            throughputs.append(num_requests / time_taken_all)

async def get_user_preds_with_threads_lkpy(user, algo, items, session, sem, times, model):
    async with sem:  # semaphore limits num of simultaneous downloads
        return await get_user_preds_threads_lkpy(user, algo, items, session, times, model)        
        
async def get_user_preds_threads_lkpy(user, algo, items, session, times, model):
    try:
        start = perf_counter()
        results = []
        df_preds = model.predict_for_user(user, items.split(','))
        for index, value in df_preds.iteritems():
            if not math.isnan(value):
                results.append({'item': index, 'score': value})
                
        time_taken = perf_counter() - start
        times.append(time_taken)
        return results
    except:
        print(f"Unexpected preds error for user: {user}, with items: {items}. Error: {sys.exc_info()[0]}")
        raise        

#### Train models

In [ ]:
import train_save_model
algos = "bias, biasedmf" #, itemitem"
train_save_model.save_models(algos)

In [ ]:
lk_recserver_algos = ['bias', 'biasedmf', 'itemitem']
for lk_recserver_algo in lk_recserver_algos:
    print(f'Algo: {lk_recserver_algo}')
    print('Lenskit performance:')
    model = load_for_shared_mem(f'{lk_recserver_algo}.bpk')
    file_name = f'lkpy_parallel_threads_8_{lk_recserver_algo}__num_req_{num_requests}.csv'
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(get_preds_threads_lkpy(8, model, file_name))
    loop.run_until_complete(future)
    #plot_numbers(file_name)
    #hist_numbers(file_name)
    print('------------------')    
    warm_up(algo, 8, False)
    print('Recommendation server performance:')
    file_name = f'preds_{lk_recserver_algo}_parallel_threads_8_workers_4_num_req_{num_requests}.csv'
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(get_preds_sem(8, algo, file_name, True))
    loop.run_until_complete(future)
    #plot_numbers(file_name)
    #hist_numbers(file_name)
    print('*******************************************************')    
